## SVM应用的一般框架
### （1）收集数据：可以使用任意方法收集数据
### （2）准备数据：需要数值型数据
### （3）分析数据：有助于可视化分隔超平面
### （4）训练算法：SVM的大部分时间都源自训练，该过程主要实现两个参数的优化。
### （5）测试算法：十分简单的计算过程就可以实现
### （6）使用算法：几乎所有分类问题都可以使用SVM，值得一提的是，SVM本身是一个二类分类器，对多类问题应用SVM需要对代码做一些修改

### 分类器求解的优化问题

###  优化目标：找到离分隔超平面最近的点，确保它们离分隔面的距离尽可能远  $arg \underset{w,b}{max}\left \{ \underset{n}{min}(label \cdot (w^{T}))\cdot \frac{1}{\left \| w \right \|}) \right \}$

### 上述优化问然后题中，给定了一些约束条件然后求最优值，因此该问题是一个带约束条件的优化问题，对于这类优化问题引入拉格朗日乘子法，优化目标函数最后可以写成

#### &emsp;&emsp; $\underset{a}{max}\left [ \sum_{i=1}^{m} \alpha -\frac{1}{2}\sum_{i,j=1}^{m}label^{(i)}\cdot label^{(j)}\cdot \alpha {_{i}}\cdot \alpha {_{j}}\left \langle x^{_{(i)}}, x^{_{(j)}}  \right \rangle\right ]$

### 其约束条件为：

#### &emsp;&emsp;  $\alpha \geqslant 0 $ ，和 $ \sum_{i=1}^{m} \alpha {_{i}}\cdot label^{(i)}$

### 引入松弛变量（stack variable），允许有些数据点可以处以分隔面的错误一侧，此时新的约束条件为

#### &emsp;&emsp;  $C\geqslant \alpha \geqslant 0 $ ，和 $ \sum_{i=1}^{m} \alpha {_{i}}\cdot label^{(i)}$

### 这里的常数C用于控制“最大化间隔” 和 “保证大部分点的函数间隔小于1.0”这两个目标的权重，常数C是一个参数，可以通过调节该参数得到不同的结果，一旦求出所有的alpha ,那么分隔超平面就可以通过这些alpha来表达

### 程序清单6-1 SMO算法中的辅助函数   SMO算法没有理解透彻，

In [98]:
# https://www.cnblogs.com/harvey888/p/5574461. 讲解SMO算法很清楚的帖子
# https://blog.csdn.net/on2way/article/details/47730367  SMO算法原理与实战求解  带后续回头再理解
# https://blog.csdn.net/on2way/article/details/47729419
from numpy import *
 
def loadDataSet(filename):
    dataMat=[]
    labelMat=[]
    fr=open(filename)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat
 
def selectJrand(i,m):
    #i是第一个alpha的下标，m都是alpha的总个数
    j=i
    while(j==i):
        #j是第二个alpha的下标
        #如果j与i相同，则重新选取一个
        j=int(random.uniform(0,m))
    return j
 
def clipAlpha(aj,H,L):
    #设置上下界限
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj
dataArr,labelArr = loadDataSet('testSest.txt')

### SMO函数的伪代码
####  创建1个alpha向量并将其初始化为0向量
####  当迭代次数小于最大迭代次数时（外循环）：
####  &emsp;&emsp;对数据集中的每个数据向量（内循环）：
#### &emsp;&emsp; &emsp;&emsp; 如果该数据向量可以被优化：
#### &emsp;&emsp; &emsp;&emsp;&emsp;&emsp; 随机选择另一个数据向量：
#### &emsp;&emsp; &emsp;&emsp;&emsp;&emsp; 同时优化这两个向量
#### &emsp;&emsp; &emsp;&emsp;&emsp;&emsp; 如果两个向量都不能被优化，退出内循环
#### &emsp;&emsp; 如果所有向量都没被优化，增加迭代数目，继续下一次循环

### 程序清单6-2  简化版SMO算法

In [99]:

#函数输入：数据、标签集、常数、容错率、最大循环次数
def smoSimple(dataMatIn,classlabels,C,toler,maxIter):
    dataMatrix=mat(dataMatIn)     # 转换成矩阵
    labelMat=mat(classlabels).transpose() # 转换成矩阵，并转置，标记成为一个列向量，每一行和数据矩阵对应
    b=0# 参数b的初始化
    m,n=shape(dataMatrix)# 行(样本数)，列 
    #m表示样本个数，n表示特征维度    
    alphas=mat(zeros((m,1)))# 参数alphas是个list，初始化也是全0，大小等于样本数
    iter=0 # 当前迭代次数，maxIter是最大迭代次数 
    
    while(iter<maxIter):
    #外循环
        alphaPairsChanged=0  # 表示alpha对是否更新，=0 表示未更新
        for i in range(m):# 【外循环】，选第一个alpha
            # 第i样本的预测类别, f(xi)=w xi+b
            # multiply(a,b)就是个乘法,如果a,b是两个长度为N的数组,那么对应元素相乘,得到的仍是一个长度为N的数组 
            # dataMatrix: N*p, 
            # multiply(alphas,labelMat).T: 1*N
            # dataMatrix*dataMatrix[i,:].T: N*1
            fXi=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b
            #multiply是numpy的乘法函数，.T是转置
            #第i个样本对应的判别结果
            #(dataMatrix*dataMatrix[i,:].T)是一个核函数计算
            Ei=fXi-float(labelMat[i])
            #误差，预测结果与真实类别值
            #是否可以继续优化--alpha(i)违反KKT条件，继续优化
            if((labelMat[i]*Ei<-toler)and (alphas[i]<C))or ((labelMat[i]*Ei>toler)and (alphas[i]>0)):
                #如果误差大于容错率或者alpha值不符合约束，则进入优化
                # 【内循环】，选择第j个样本
                # 第j个样本的预测类别
                j=selectJrand(i,m)
                #随机选择第二个alpha
                fXj=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                #计算第二个alpha的值
                Ej=fXj-float(labelMat[j])
                #得到两个样本对应的两个alpha对应的误差值
                alphaIold=alphas[i].copy()# 拷贝,分配新的内存,旧的alpha[i]的值
                alphaJold=alphas[j].copy()# 拷贝,分配新的内存,旧的alpha[j]的值
                
                #更新L和H, alpha(j)的范围
                #存储原本的alpha值
                if(labelMat[i]!=labelMat[j]):
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])
                if L==H:
                    print('L=H')
                    continue
                
                #更新alpha(j)
                #计算上下阈值
                #针对y1,y2的值相同与否，上下值也不同
                eta=2.0*dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:].T
                #最优修改量
                if eta>=0:
                    print('eta>=0')
                    continue
                alphas[j]-=labelMat[j]*(Ei-Ej)/eta
                #更新alpha
                alphas[j]=clipAlpha(alphas[j],H,L) # 门限函数将alpha(j)调整到[L,H]之间
                if (abs(alphas[j]-alphaJold)<0.00001): #如果alpha(j)修改量很微小
                    print('j is not moving enough')
                    continue
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j]) #更新alpha[i],由一个alpha确定另一个alpha
                
                #重新计算b
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-labelMat[j]*(alphas[j]-alphaJold)\
                *dataMatrix[i,:]*dataMatrix[j,:].T
                b2=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T-labelMat[j]*(alphas[j]-alphaJold)\
                *dataMatrix[j,:]*dataMatrix[j,:].T
                #更新两个b值
                if (0<alphas[i])and(C>alphas[i]):
                    b=b1
                elif (0<alphas[j])and(C>alphas[j]):
                    b=b2
                else:
                    b=(b1+b2)/2.0
                alphaPairsChanged+=1   # 发生改变的 alpha对 的数目
                print('iter:%d i:%d,pairs changed %d'%(iter,i,alphaPairsChanged))
        #如果 对于整个for循环（ i=1,...,m ）, 所有alpha对 都没有更新，++iter
        #这一过程连续地进行 maxIter次，while循环才结束
        if(alphaPairsChanged==0):
            iter += 1
        #如果 对于整个for循环（ i=1,...,m ）, 某个alpha对 有更新，那么迭代次数归0，重新优化（while循环中iter=0）
        else:
            iter = 0
        print('iteration number : %d' %iter)
    return b,alphas
# 只有当某次优化更新达到了最大迭代次数，这个时候才返回优化之后的alpha和b
b,alphas=smoSimple(dataArr,labelArr,0.6,0.001,40)     


L=H
iter:0 i:1,pairs changed 1
L=H
j is not moving enough
L=H
j is not moving enough
L=H
L=H
j is not moving enough
j is not moving enough
L=H
L=H
iter:0 i:23,pairs changed 2
L=H
L=H
j is not moving enough
j is not moving enough
iter:0 i:54,pairs changed 3
j is not moving enough
iteration number : 0
iter:0 i:1,pairs changed 1
j is not moving enough
j is not moving enough
L=H
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
L=H
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
L=H
j is not moving enough
j is not moving enough
L=H
j is not moving enough
iteration number : 0
j is not moving enough
L=H
L=H
j is not moving enough
iter:0 i:23,pairs changed 1
j is not moving enough
iter:0 i:39,pairs changed 2
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
L=H
L=H
j is not moving enough
j is not moving enough
j is not mo

j is not moving enough
j is not moving enough
iteration number : 9
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 10
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 11
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 12
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 13
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 14
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 15
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 16
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 17
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 18
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 19
j is not moving enough


j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 24
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 25
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 26
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 27
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 28
j is not moving enough
iter:28 i:52,pairs changed 1
j is not moving enough
j is not moving enough
iteration number : 0
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 1
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 2
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 3
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 4
j is not moving enough
j is not moving enough
j is not moving enou

iteration number : 2
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 3
j is not moving enough
j is not moving enough
iter:3 i:55,pairs changed 1
iteration number : 0
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 1
j is not moving enough
j is not moving enough
L=H
j is not moving enough
iteration number : 2
j is not moving enough
j is not moving enough
L=H
j is not moving enough
iteration number : 3
j is not moving enough
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 4
j is not moving enough
j is not moving enough
L=H
j is not moving enough
iteration number : 5
j is not moving enough
j is not moving enough
j is not moving enough
iter:5 i:54,pairs changed 1
j is not moving enough
iteration number : 0
j is not moving enough
j is not moving enough
j is not moving enough
iteration number : 1
j is not moving enough
j is not moving enough
j is not 

j is not moving enough
iter:28 i:55,pairs changed 1
iteration number : 0
j is not moving enough
j is not moving enough
iteration number : 1
j is not moving enough
j is not moving enough
iteration number : 2
j is not moving enough
j is not moving enough
iteration number : 3
j is not moving enough
j is not moving enough
iteration number : 4
j is not moving enough
j is not moving enough
iteration number : 5
j is not moving enough
j is not moving enough
iteration number : 6
j is not moving enough
j is not moving enough
iteration number : 7
j is not moving enough
j is not moving enough
iteration number : 8
j is not moving enough
j is not moving enough
iteration number : 9
j is not moving enough
j is not moving enough
iteration number : 10
j is not moving enough
j is not moving enough
iteration number : 11
j is not moving enough
j is not moving enough
iteration number : 12
j is not moving enough
j is not moving enough
iteration number : 13
j is not moving enough
j is not moving enough
iterat

In [100]:
print(b)
print(alphas)

[[-3.8619747]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.07118549]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.28956024]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.05701973]
 [ 0.303726  ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [101]:
print(shape(alphas[alphas>0]))

(1, 4)


In [102]:
for i in range(100):
    if alphas[i]>0.0:
        print(dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0


### 程序清单6-3  完整版的Platt SMO的支持函数

Platt SMO算法是通过一个外循环来选择第一个alpha值，且选择过程会在两种方式之间进行交替。 
第一种方式是：在所有的数据集上进行单独扫描，另外一种方式就是在非边界的alpha中实现单遍扫描。（其中非边界的alpha是指那些不等于边界0或者C或者alpha的值）

我们在使用第二种方法的时候，也就是非边界的alpha值扫描的时候，首先需要建立这些alpha值的列表，然后对这个表进行遍历。同时，该步骤会跳过 那些已知的不会改变的alpha的值。

在选择第一个alpha的值后，算法会通过一个内循环来选择一个第二个alpha的值。在优化过程中，会通过最大化步长的方式来获得第二个alpha值。 
我们会建立一个全局的缓存用于保存误差值，从中选择使用步长或者Ei - Ej 最大的alpha值


In [103]:
import numpy as np
# 完整版的Platt SMO算法
'''
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]                    
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        #误差缓存
        self.eCache = np.mat(np.zeros((self.m, 2)))

def calcEk(oS,k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
    #内循环中的启发式方法
    
def selectJ(i,oS,Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]
    #构建了一个非零表
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i:continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                #选择具有最大步长的j
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK,Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]
'''



#### 1.建立一个数据结构来保存重要的值，通过一个对象来完成。最为一个数据结构来使用对象。 那就是我们将值传给函数的时候，可以通过将所有的数据移到一个结构中来实现，这样可以省掉手工输入的麻烦。数据可以通过一个对象来进行传递。为达到这个目的，我们构建一个仅包含有init方法的optStruct类。 
#### 2.辅助函数calcEk()能够计算E值并返回，且才有内嵌式的方式来完成的。 
#### 3.函数selectJ()用于选择第二个alpha或者内循环的alpha值。 
#### （1）将输入值Ei 在缓存中设置成为有效的。 
#### （2）在eCache中，nonzero(oS.eCache[:,0].A)[0]构建了一个非零表 
#### （3）nonzero()语句返回的是非零E值所对应的alpha值，而不是E值本身。程序会在所有的值进行选号并选择其中使得改变最大的那个值。如果是第一次循环，就随机选择。 
#### 4.辅助函数updateEk() 会计算误差值并存入缓存当中，在对于alpha值进行优化的时候回用到这个值。


In [104]:
# 完整的Platt SMO算法中的优化例程
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        #第二个alpha选择中的启发式方法
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C,oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: 
            print("L==H"); return 0
        eta = 2.0 * oS.X[i,:] * oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - oS.X[j,:]*oS.X[j,:].T
        if eta >= 0:
            print("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        #更新误差缓存
        updateEk(oS,j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        #更新误差缓存
        updateEk(oS,i)
        b1 = oS.b-Ei-oS.labelMat[i]*(oS.alphas[i] - alphaIold)*oS.X[i,:]*oS.X[i,:].T-oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b-Ej-oS.labelMat[i]*(oS.alphas[i] - alphaIold)*oS.X[i,:]*oS.X[j,:].T-oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0< oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0< oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        return 1
    else:
        return 0
    """
    使用了自己的数据结构，该结构的在参数oS中传递
    """
# 1.上面的程序与函数smoSimple()几乎一样，只是使用了自己的数据结构。 
# 2.使用函数selectJ()来选择第二个alpha的值。 
# 3.当alpha的值改变的时候就更新Ecache 。

In [70]:
# 完整版platt SMO的外循环代码

def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup):   # 将默认参数改为参数调用，否则后面调试通不过，改了半天代码错误
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(),C,toler,kTup)
    iter = 0
    entireSet = True; aphaPairsChanged = 0
    while (iter < maxIter) and ((aphaPairsChanged > 0) or (entireSet)):
        aphaPairsChanged = 0
        #遍历数据集中所有的值
        if entireSet:
            for i in range(oS.m):
                aphaPairsChanged += innerL(i,oS)
                print("fullSet, iter:  %d i,%d, pairs changed %d" %(iter,i,aphaPairsChanged))
            iter += 1
        #遍历非边界值
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0)*(oS.alphas.A < C))[0]
            for i in nonBoundIs:
                aphaPairsChanged += innerL(i, oS)
                print("non-bound, iter:  %d i,%d, pairs changed %d" %(iter,i,aphaPairsChanged))
                iter += 1
        if entireSet : 
            entireSet = False
        elif (aphaPairsChanged == 0): 
                entireSet = True
        print("iteration number: %d " % iter)
    return oS.b, oS.alphas

#1.上述函数smoP()与函数smoSimple()大致一样，在输入方面是一样的。 
#2.函数主体是while循环，循环的退出条件更加多一些，迭代次数超过指定的最大值，或者遍历整个集合都未对alpha进行修改，就会退出循环。 
#3.变量maxIter ，这里一次迭代定义为一次循环过程，而不管循环具体做了什么事。 
#4.while循环下面一开始的for循环在数据集上遍历任意可能的alpha。通过调用innerL()函数来选择第二个alpha ，
#并在可能时对其优化处理，若有任意一堆alpha值发生改变，就返回1 


In [127]:
b,alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40,('lin',0))

fullSet, iter:  0 i,0, pairs changed 1
fullSet, iter:  0 i,1, pairs changed 1
eta>=0
fullSet, iter:  0 i,2, pairs changed 1
fullSet, iter:  0 i,3, pairs changed 1
eta>=0
fullSet, iter:  0 i,4, pairs changed 1
eta>=0
fullSet, iter:  0 i,5, pairs changed 1
eta>=0
fullSet, iter:  0 i,6, pairs changed 1
fullSet, iter:  0 i,7, pairs changed 1
fullSet, iter:  0 i,8, pairs changed 2
fullSet, iter:  0 i,9, pairs changed 2
fullSet, iter:  0 i,10, pairs changed 2
fullSet, iter:  0 i,11, pairs changed 2
fullSet, iter:  0 i,12, pairs changed 2
fullSet, iter:  0 i,13, pairs changed 3
fullSet, iter:  0 i,14, pairs changed 4
fullSet, iter:  0 i,15, pairs changed 5
fullSet, iter:  0 i,16, pairs changed 5
fullSet, iter:  0 i,17, pairs changed 6
L==H
fullSet, iter:  0 i,18, pairs changed 6
fullSet, iter:  0 i,19, pairs changed 6
fullSet, iter:  0 i,20, pairs changed 6
fullSet, iter:  0 i,21, pairs changed 6
fullSet, iter:  0 i,22, pairs changed 6
fullSet, iter:  0 i,23, pairs changed 7
fullSet, iter:  0

In [128]:
 alphas[alphas>0]
    

matrix([[ 0.03883508,  0.02894415,  0.08146528,  0.02820373,  0.010795  ,
          0.00908251,  0.02894415,  0.03728625,  0.0534252 ]])

In [129]:
b

matrix([[-2.61551822]])

我们计算了alpha之后，现在如何进行分类呢？首先必须基于alpha值得到超平面，包括了w值。 
根据下面的函数来进行：

In [76]:

def calcWs(alphas, dataArr, classLabels):
    dataArr,labelArr = loadDataSet('testSest.txt')
    X = np.mat(dataArr); labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i], X[i, :].T)
    return w


In [108]:
ws=calcWs(alphas,dataArr,labelArr)

In [109]:
print(ws)

[[ 0.81565041]
 [-0.23311149]]


In [110]:
datMat = mat(dataArr)
datMat[0]*mat(ws)+b

matrix([[-1.43349956]])

In [111]:
labelArr[0]

-1.0

In [112]:
datMat[1]*mat(ws)+b

matrix([[-1.99554089]])

In [113]:
labelArr[1]

-1.0

In [114]:
datMat[2]*mat(ws)+b

matrix([[ 2.66574065]])

In [115]:
labelArr[2]

1.0

### 核函数（kernel） ：将数据转换成易于分类器理解的形式，径向基函数（radial basis function）最流行的核函数，应用于分类器，将低维特征空间映射到高维空间

### 径向基函数的高斯版本：$k(x,y)=exp\left ( \frac{-\left \| x-y \right \|^{2}}{2\delta ^{2}} \right )$ 

### $\delta$是用户定义的用于确定到达率reach或者说函数值跌到0的速度参数

### 程序清单6-6 核转换函数

In [116]:

def kernelTrans(X,A,kTup):
    #kTup是一个包含核函数信息的元组
    m,n=shape(X)
    K=mat(zeros((m,1)))
    if kTup[0]=='lin':K=X*A.T
    #最简单的内积方式
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow=X[j,:]-A
            K[j]=deltaRow*deltaRow.T
        K=exp(K/(-1*kTup[1]**2))
    else:
        raise NameError('Houston We have a Problem -- That Kernel is not recognized')
    return K

#这是一个核函数转换的函数。
#kTup包含核函数的信息，第一个参数表示核函数类型，第二个是核函数的参数。上面的函数提供了两种核函数方法。
#1、内积，计算所有数据集和数据集中的一行之间的展开。
#2、for循环对每个元素计算高斯函数的值，for循环后，将计算过程应用的整个向量中去。
#deltaRow表示的是|x-y|
 

class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn   # 输入数据
        self.labelMat = classLabels# 标记
        self.C = C#常数C,目标函数正则化项系数
        self.tol = toler#容错率
        self.m = np.shape(dataMatIn)[0]#样本数
        #记住是双括号
        self.alphas = np.mat(np.zeros((self.m, 1)))# 参数alphas是个list，初始化也是全0，大小等于样本数
        self.b = 0#参数b的初始化
        self.eCache = np.mat(np.zeros((self.m, 2)))#存放误差E,第i行：第1列是标志位，第2列是Ei
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):#对K逐列处理
            self.K[: , i] = kernelTrans(self.X, self.X[i, :], kTup)
            
#相比原本的数据结构，增加了核函数的信息。
#K是先构造矩阵，然后通过循环将数据进行转换填充进去。
#然后将InnerL和calcEk函数里头的内积替换成核函数转换后的结果。


### 程序清单6-7 使用核函数对innerL（） 及 calcEk()函数进行修改

In [122]:
def innerL(i,oS):
    Ei=calcEk(oS,i)
    if((oS.labelMat[i]*Ei<-oS.tol)and (oS.alphas[i]<oS.C))or((oS.labelMat[i]*Ei>oS.tol)and(oS.alphas[i]>0)):
        j,Ej=selectJ(i,oS,Ei)
        #启发式方法选择第二个alpha
        alphaIold=oS.alphas[i].copy()
        alphaJold=oS.alphas[j].copy()
        if(oS.labelMat[i]!=oS.labelMat[j]):
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[j]+oS.alphas[i])
        if L==H :
            print('L==H')
            return 0
        eta=2.0*oS.K[i,j]-oS.K[i,j].T-oS.K[j,j]
        #计算eta
        if eta>=0:
            print('eta>=0')
            return 0
        oS.alphas[j]-=oS.labelMat[j]*(Ei-Ej)/eta
        #计算未经剪辑的最优解
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        #增加约束条件
        updateEk(oS,j)
        #更新对应的误差值
        if(abs(oS.alphas[j]-alphaJold)<0.00001):
            print('j is not moving enough')
            return 0
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        #由第二个alpha求解另一个alpha
        updateEk(oS,i)
        b1=oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]\
                   -oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2=oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]\
                   -oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if(0<oS.alphas[i]) and (oS.C>oS.alphas[i]):
            oS.b=b1
        elif(0<oS.alphas[j])and(oS.C>oS.alphas[j]):
            oS.b=b2
        else:
            oS.b=(b1+b2)/2.0
        return 1
    else:
        return 0


In [119]:
def calcEk(oS,k):
    #用来计算误差的函数
    fXk=float(multiply(oS.alphas,oS.labelMat).T*(oS.K[:,k]))+oS.b
    Ek=fXk-float(oS.labelMat[k])
    return Ek

### 程序清单6-8 利用核函数进行分类的径向基测试函数

In [120]:
#利用核函数进行分类的径向基测试函数
def testRbf(k1=1.3):
    dataArr,labelArr=loadDataSet('testSetRBF.txt')
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,('rbf',k1))
    datMat=mat(dataArr)
    labelMat=mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd]
    labelSV=labelMat[svInd]
    print('there are %d support vectors' %shape(sVs)[0])
    m,n=shape(datMat)
    errCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,datMat[i,:],('rbf',k1))
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errCount+=1
    print('the training error rate is %f' %(float(errCount)/m))
    dataArr,labelArr=loadDataSet('testSetRBF2.txt')
    errCount=0
    datMat=mat(dataArr)
    labelMat=mat(labelArr).transpose()
    m,n=shape(dataArr)
    for i in range(m):
        kernelEval=kernelTrans(sVs,datMat[i,:],('rbf',k1))
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errCount+=1
    print('the training error rate is %f' %(float(errCount)/m))


In [121]:
testRbf()

L==H
fullSet, iter:  0 i,0, pairs changed 0
fullSet, iter:  0 i,1, pairs changed 1
fullSet, iter:  0 i,2, pairs changed 1
j is not moving enough
fullSet, iter:  0 i,3, pairs changed 1
fullSet, iter:  0 i,4, pairs changed 1
fullSet, iter:  0 i,5, pairs changed 2
fullSet, iter:  0 i,6, pairs changed 3
fullSet, iter:  0 i,7, pairs changed 3
fullSet, iter:  0 i,8, pairs changed 4
fullSet, iter:  0 i,9, pairs changed 4
fullSet, iter:  0 i,10, pairs changed 4
L==H
fullSet, iter:  0 i,11, pairs changed 4
fullSet, iter:  0 i,12, pairs changed 4
fullSet, iter:  0 i,13, pairs changed 4
fullSet, iter:  0 i,14, pairs changed 4
fullSet, iter:  0 i,15, pairs changed 5
fullSet, iter:  0 i,16, pairs changed 6
fullSet, iter:  0 i,17, pairs changed 6
fullSet, iter:  0 i,18, pairs changed 7
fullSet, iter:  0 i,19, pairs changed 8
fullSet, iter:  0 i,20, pairs changed 8
fullSet, iter:  0 i,21, pairs changed 8
fullSet, iter:  0 i,22, pairs changed 8
j is not moving enough
fullSet, iter:  0 i,23, pairs chan

fullSet, iter:  44 i,44, pairs changed 3
j is not moving enough
fullSet, iter:  44 i,45, pairs changed 3
fullSet, iter:  44 i,46, pairs changed 3
fullSet, iter:  44 i,47, pairs changed 3
L==H
fullSet, iter:  44 i,48, pairs changed 3
fullSet, iter:  44 i,49, pairs changed 3
fullSet, iter:  44 i,50, pairs changed 3
fullSet, iter:  44 i,51, pairs changed 3
fullSet, iter:  44 i,52, pairs changed 3
fullSet, iter:  44 i,53, pairs changed 4
j is not moving enough
fullSet, iter:  44 i,54, pairs changed 4
j is not moving enough
fullSet, iter:  44 i,55, pairs changed 4
j is not moving enough
fullSet, iter:  44 i,56, pairs changed 4
fullSet, iter:  44 i,57, pairs changed 4
j is not moving enough
fullSet, iter:  44 i,58, pairs changed 4
fullSet, iter:  44 i,59, pairs changed 5
j is not moving enough
fullSet, iter:  44 i,60, pairs changed 5
j is not moving enough
fullSet, iter:  44 i,61, pairs changed 5
j is not moving enough
fullSet, iter:  44 i,62, pairs changed 5
fullSet, iter:  44 i,63, pairs c

###  基于SVM的数字识别
### （1）收集数据：提供的文本文件
### （2）准备数据：基于二值图像构造向量
### （3）分析数据：对图像向量进行目测
### （4）训练算法：采用两种不同的核函数，并对径向基核函数采用不同的设置来运行SMO算法
### （5）测试算法：编写一个函数测试不同的核函数并计算错误率
### （6）使用算法：一个图像识别的完整应用还需要一些图像处理的知识

### 程序清单6-9 基于SVM的手写数字识别

In [125]:
def img2vector(filename):
    """
    Function：   32*32图像转换为1*1024向量

    Input：      filename：文件名称字符串

    Output： returnVect：转换之后的1*1024向量
    """ 
    #初始化要返回的1*1024向量
    returnVect = zeros((1, 1024))
    #打开文件
    fr = open(filename)
    #读取文件信息
    for i in range(32):
        #循环读取文件的前32行
        lineStr = fr.readline()
        for j in range(32):
            #将每行的头32个字符存储到要返回的向量中
            returnVect[0, 32*i+j] = int(lineStr[j])
    #返回要输出的1*1024向量
    return returnVect

def loadImages(dirName):
    """
    Function：   加载图片

    Input：      dirName：文件路径

    Output： trainingMat：训练数据集
                hwLabels：数据标签
    """ 
    from os import listdir
    #初始化数据标签
    hwLabels = []
    #读取文件列表
    trainingFileList = listdir(dirName)
    #读取文件个数
    m = len(trainingFileList)
    #初始化训练数据集
    trainingMat = zeros((m,1024))
    #填充数据集
    for i in range(m):
        #遍历所有文件
        fileNameStr = trainingFileList[i]
        #提取文件名称
        fileStr = fileNameStr.split('.')[0]
        #提取数字标识
        classNumStr = int(fileStr.split('_')[0])
        #数字9记为-1类
        if classNumStr == 9: hwLabels.append(-1)
        #其他数字记为+1类
        else: hwLabels.append(1)
        #提取图像向量，填充数据集
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    #返回数据集和数据标签
    return trainingMat, hwLabels

def testDigits(kTup = ('rbf',10)):
    """
    Function：   手写数字分类函数

    Input：      kTup：核函数采用径向基函数

    Output： 输出打印信息
    """ 
    #导入数据集
    dataArr, labelArr = loadImages('trainingDigits')
    #调用Platt SMO算法
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    #初始化数据矩阵和标签向量
    datMat = mat(dataArr); labelMat = mat(labelArr).transpose()
    #记录支持向量序号
    svInd = nonzero(alphas.A > 0)[0]
    #读取支持向量
    sVs = datMat[svInd]
    #读取支持向量对应标签
    labelSV = labelMat[svInd]
    #输出打印信息
    print("there are %d Support Vectors" % shape(sVs)[0])
    #获取数据集行列值
    m, n = shape(datMat)
    #初始化误差计数
    errorCount = 0
    #遍历每一行，利用核函数对训练集进行分类
    for i in range(m):
        #利用核函数转换数据
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        #仅用支持向量预测分类
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        #预测分类结果与标签不符则错误计数加一
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    #打印输出分类错误率
    print("the training error rate is: %f" % (float(errorCount)/m))
    #导入测试数据集
    dataArr,labelArr = loadImages('testDigits')
    #初始化误差计数
    errorCount = 0
    #初始化数据矩阵和标签向量
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    #获取数据集行列值
    m,n = shape(datMat)
    #遍历每一行，利用核函数对测试集进行分类
    for i in range(m):
        #利用核函数转换数据
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        #仅用支持向量预测分类
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        #预测分类结果与标签不符则错误计数加一
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    #打印输出分类错误率
    print("the test error rate is: %f" % (float(errorCount)/m))

testDigits(('rbf',20))

L==H
fullSet, iter:  0 i,0, pairs changed 0
fullSet, iter:  0 i,1, pairs changed 1
fullSet, iter:  0 i,2, pairs changed 2
fullSet, iter:  0 i,3, pairs changed 3
j is not moving enough
fullSet, iter:  0 i,4, pairs changed 3
j is not moving enough
fullSet, iter:  0 i,5, pairs changed 3
j is not moving enough
fullSet, iter:  0 i,6, pairs changed 3
fullSet, iter:  0 i,7, pairs changed 4
fullSet, iter:  0 i,8, pairs changed 5
fullSet, iter:  0 i,9, pairs changed 6
fullSet, iter:  0 i,10, pairs changed 7
fullSet, iter:  0 i,11, pairs changed 8
j is not moving enough
fullSet, iter:  0 i,12, pairs changed 8
fullSet, iter:  0 i,13, pairs changed 9
fullSet, iter:  0 i,14, pairs changed 10
fullSet, iter:  0 i,15, pairs changed 11
fullSet, iter:  0 i,16, pairs changed 12
fullSet, iter:  0 i,17, pairs changed 13
fullSet, iter:  0 i,18, pairs changed 14
fullSet, iter:  0 i,19, pairs changed 15
j is not moving enough
fullSet, iter:  0 i,20, pairs changed 15
fullSet, iter:  0 i,21, pairs changed 16
L=

L==H
fullSet, iter:  0 i,183, pairs changed 16
L==H
fullSet, iter:  0 i,184, pairs changed 16
L==H
fullSet, iter:  0 i,185, pairs changed 16
L==H
fullSet, iter:  0 i,186, pairs changed 16
L==H
fullSet, iter:  0 i,187, pairs changed 16
L==H
fullSet, iter:  0 i,188, pairs changed 16
L==H
fullSet, iter:  0 i,189, pairs changed 16
L==H
fullSet, iter:  0 i,190, pairs changed 16
L==H
fullSet, iter:  0 i,191, pairs changed 16
L==H
fullSet, iter:  0 i,192, pairs changed 16
L==H
fullSet, iter:  0 i,193, pairs changed 16
L==H
fullSet, iter:  0 i,194, pairs changed 16
L==H
fullSet, iter:  0 i,195, pairs changed 16
L==H
fullSet, iter:  0 i,196, pairs changed 16
L==H
fullSet, iter:  0 i,197, pairs changed 16
L==H
fullSet, iter:  0 i,198, pairs changed 16
L==H
fullSet, iter:  0 i,199, pairs changed 16
L==H
fullSet, iter:  0 i,200, pairs changed 16
L==H
fullSet, iter:  0 i,201, pairs changed 16
L==H
fullSet, iter:  0 i,202, pairs changed 16
L==H
fullSet, iter:  0 i,203, pairs changed 16
L==H
fullSet,

L==H
fullSet, iter:  0 i,359, pairs changed 16
L==H
fullSet, iter:  0 i,360, pairs changed 16
L==H
fullSet, iter:  0 i,361, pairs changed 16
L==H
fullSet, iter:  0 i,362, pairs changed 16
L==H
fullSet, iter:  0 i,363, pairs changed 16
L==H
fullSet, iter:  0 i,364, pairs changed 16
L==H
fullSet, iter:  0 i,365, pairs changed 16
L==H
fullSet, iter:  0 i,366, pairs changed 16
L==H
fullSet, iter:  0 i,367, pairs changed 16
L==H
fullSet, iter:  0 i,368, pairs changed 16
L==H
fullSet, iter:  0 i,369, pairs changed 16
L==H
fullSet, iter:  0 i,370, pairs changed 16
L==H
fullSet, iter:  0 i,371, pairs changed 16
L==H
fullSet, iter:  0 i,372, pairs changed 16
L==H
fullSet, iter:  0 i,373, pairs changed 16
L==H
fullSet, iter:  0 i,374, pairs changed 16
L==H
fullSet, iter:  0 i,375, pairs changed 16
L==H
fullSet, iter:  0 i,376, pairs changed 16
L==H
fullSet, iter:  0 i,377, pairs changed 16
L==H
fullSet, iter:  0 i,378, pairs changed 16
L==H
fullSet, iter:  0 i,379, pairs changed 16
L==H
fullSet,

L==H
fullSet, iter:  0 i,536, pairs changed 16
L==H
fullSet, iter:  0 i,537, pairs changed 16
L==H
fullSet, iter:  0 i,538, pairs changed 16
L==H
fullSet, iter:  0 i,539, pairs changed 16
L==H
fullSet, iter:  0 i,540, pairs changed 16
L==H
fullSet, iter:  0 i,541, pairs changed 16
L==H
fullSet, iter:  0 i,542, pairs changed 16
L==H
fullSet, iter:  0 i,543, pairs changed 16
L==H
fullSet, iter:  0 i,544, pairs changed 16
L==H
fullSet, iter:  0 i,545, pairs changed 16
L==H
fullSet, iter:  0 i,546, pairs changed 16
L==H
fullSet, iter:  0 i,547, pairs changed 16
L==H
fullSet, iter:  0 i,548, pairs changed 16
L==H
fullSet, iter:  0 i,549, pairs changed 16
L==H
fullSet, iter:  0 i,550, pairs changed 16
L==H
fullSet, iter:  0 i,551, pairs changed 16
L==H
fullSet, iter:  0 i,552, pairs changed 16
L==H
fullSet, iter:  0 i,553, pairs changed 16
L==H
fullSet, iter:  0 i,554, pairs changed 16
L==H
fullSet, iter:  0 i,555, pairs changed 16
L==H
fullSet, iter:  0 i,556, pairs changed 16
L==H
fullSet,

L==H
fullSet, iter:  0 i,713, pairs changed 16
L==H
fullSet, iter:  0 i,714, pairs changed 16
L==H
fullSet, iter:  0 i,715, pairs changed 16
L==H
fullSet, iter:  0 i,716, pairs changed 16
L==H
fullSet, iter:  0 i,717, pairs changed 16
L==H
fullSet, iter:  0 i,718, pairs changed 16
L==H
fullSet, iter:  0 i,719, pairs changed 16
L==H
fullSet, iter:  0 i,720, pairs changed 16
L==H
fullSet, iter:  0 i,721, pairs changed 16
L==H
fullSet, iter:  0 i,722, pairs changed 16
L==H
fullSet, iter:  0 i,723, pairs changed 16
L==H
fullSet, iter:  0 i,724, pairs changed 16
L==H
fullSet, iter:  0 i,725, pairs changed 16
L==H
fullSet, iter:  0 i,726, pairs changed 16
L==H
fullSet, iter:  0 i,727, pairs changed 16
L==H
fullSet, iter:  0 i,728, pairs changed 16
L==H
fullSet, iter:  0 i,729, pairs changed 16
L==H
fullSet, iter:  0 i,730, pairs changed 16
L==H
fullSet, iter:  0 i,731, pairs changed 16
L==H
fullSet, iter:  0 i,732, pairs changed 16
L==H
fullSet, iter:  0 i,733, pairs changed 16
L==H
fullSet,

...

j is not moving enough
fullSet, iter:  181 i,1024, pairs changed 0
fullSet, iter:  181 i,1025, pairs changed 0
fullSet, iter:  181 i,1026, pairs changed 0
fullSet, iter:  181 i,1027, pairs changed 0
fullSet, iter:  181 i,1028, pairs changed 0
fullSet, iter:  181 i,1029, pairs changed 0
fullSet, iter:  181 i,1030, pairs changed 0
fullSet, iter:  181 i,1031, pairs changed 0
fullSet, iter:  181 i,1032, pairs changed 0
fullSet, iter:  181 i,1033, pairs changed 0
fullSet, iter:  181 i,1034, pairs changed 0
fullSet, iter:  181 i,1035, pairs changed 0
fullSet, iter:  181 i,1036, pairs changed 0
fullSet, iter:  181 i,1037, pairs changed 0
fullSet, iter:  181 i,1038, pairs changed 0
fullSet, iter:  181 i,1039, pairs changed 0
fullSet, iter:  181 i,1040, pairs changed 0
fullSet, iter:  181 i,1041, pairs changed 0
fullSet, iter:  181 i,1042, pairs changed 0
fullSet, iter:  181 i,1043, pairs changed 0
fullSet, iter:  181 i,1044, pairs changed 0
fullSet, iter:  181 i,1045, pairs changed 0
fullSet, 

j is not moving enough
fullSet, iter:  181 i,1358, pairs changed 0
fullSet, iter:  181 i,1359, pairs changed 0
fullSet, iter:  181 i,1360, pairs changed 0
fullSet, iter:  181 i,1361, pairs changed 0
fullSet, iter:  181 i,1362, pairs changed 0
fullSet, iter:  181 i,1363, pairs changed 0
fullSet, iter:  181 i,1364, pairs changed 0
fullSet, iter:  181 i,1365, pairs changed 0
fullSet, iter:  181 i,1366, pairs changed 0
fullSet, iter:  181 i,1367, pairs changed 0
fullSet, iter:  181 i,1368, pairs changed 0
fullSet, iter:  181 i,1369, pairs changed 0
fullSet, iter:  181 i,1370, pairs changed 0
fullSet, iter:  181 i,1371, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1372, pairs changed 0
fullSet, iter:  181 i,1373, pairs changed 0
fullSet, iter:  181 i,1374, pairs changed 0
fullSet, iter:  181 i,1375, pairs changed 0
fullSet, iter:  181 i,1376, pairs changed 0
fullSet, iter:  181 i,1377, pairs changed 0
fullSet, iter:  181 i,1378, pairs changed 0
fullSet, iter:  181 i,1379, pa

j is not moving enough
fullSet, iter:  181 i,1627, pairs changed 0
fullSet, iter:  181 i,1628, pairs changed 0
fullSet, iter:  181 i,1629, pairs changed 0
fullSet, iter:  181 i,1630, pairs changed 0
fullSet, iter:  181 i,1631, pairs changed 0
fullSet, iter:  181 i,1632, pairs changed 0
fullSet, iter:  181 i,1633, pairs changed 0
fullSet, iter:  181 i,1634, pairs changed 0
fullSet, iter:  181 i,1635, pairs changed 0
fullSet, iter:  181 i,1636, pairs changed 0
fullSet, iter:  181 i,1637, pairs changed 0
fullSet, iter:  181 i,1638, pairs changed 0
fullSet, iter:  181 i,1639, pairs changed 0
fullSet, iter:  181 i,1640, pairs changed 0
fullSet, iter:  181 i,1641, pairs changed 0
fullSet, iter:  181 i,1642, pairs changed 0
fullSet, iter:  181 i,1643, pairs changed 0
fullSet, iter:  181 i,1644, pairs changed 0
fullSet, iter:  181 i,1645, pairs changed 0
fullSet, iter:  181 i,1646, pairs changed 0
fullSet, iter:  181 i,1647, pairs changed 0
fullSet, iter:  181 i,1648, pairs changed 0
fullSet, 

L==H
fullSet, iter:  181 i,1786, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1787, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1788, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1789, pairs changed 0
L==H
fullSet, iter:  181 i,1790, pairs changed 0
L==H
fullSet, iter:  181 i,1791, pairs changed 0
L==H
fullSet, iter:  181 i,1792, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1793, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1794, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1795, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1796, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1797, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1798, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1799, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1800, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1801, pairs changed 0


j is not moving enough
fullSet, iter:  181 i,1917, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1918, pairs changed 0
L==H
fullSet, iter:  181 i,1919, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1920, pairs changed 0
L==H
fullSet, iter:  181 i,1921, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1922, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1923, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1924, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1925, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1926, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1927, pairs changed 0
L==H
fullSet, iter:  181 i,1928, pairs changed 0
L==H
fullSet, iter:  181 i,1929, pairs changed 0
L==H
fullSet, iter:  181 i,1930, pairs changed 0
L==H
fullSet, iter:  181 i,1931, pairs changed 0
j is not moving enough
fullSet, iter:  181 i,1932, pairs changed 0
j is not moving enough
fullSet, iter

In [126]:
testDigits(('rbf',10))

L==H
fullSet, iter:  0 i,0, pairs changed 0
fullSet, iter:  0 i,1, pairs changed 1
fullSet, iter:  0 i,2, pairs changed 2
fullSet, iter:  0 i,3, pairs changed 3
fullSet, iter:  0 i,4, pairs changed 4
fullSet, iter:  0 i,5, pairs changed 5
fullSet, iter:  0 i,6, pairs changed 6
fullSet, iter:  0 i,7, pairs changed 7
fullSet, iter:  0 i,8, pairs changed 8
fullSet, iter:  0 i,9, pairs changed 9
fullSet, iter:  0 i,10, pairs changed 10
fullSet, iter:  0 i,11, pairs changed 11
fullSet, iter:  0 i,12, pairs changed 12
fullSet, iter:  0 i,13, pairs changed 13
fullSet, iter:  0 i,14, pairs changed 14
fullSet, iter:  0 i,15, pairs changed 15
fullSet, iter:  0 i,16, pairs changed 16
fullSet, iter:  0 i,17, pairs changed 17
fullSet, iter:  0 i,18, pairs changed 18
fullSet, iter:  0 i,19, pairs changed 19
fullSet, iter:  0 i,20, pairs changed 20
fullSet, iter:  0 i,21, pairs changed 21
fullSet, iter:  0 i,22, pairs changed 22
fullSet, iter:  0 i,23, pairs changed 23
fullSet, iter:  0 i,24, pairs c

L==H
fullSet, iter:  0 i,186, pairs changed 47
L==H
fullSet, iter:  0 i,187, pairs changed 47
L==H
fullSet, iter:  0 i,188, pairs changed 47
L==H
fullSet, iter:  0 i,189, pairs changed 47
L==H
fullSet, iter:  0 i,190, pairs changed 47
L==H
fullSet, iter:  0 i,191, pairs changed 47
L==H
fullSet, iter:  0 i,192, pairs changed 47
L==H
fullSet, iter:  0 i,193, pairs changed 47
L==H
fullSet, iter:  0 i,194, pairs changed 47
L==H
fullSet, iter:  0 i,195, pairs changed 47
L==H
fullSet, iter:  0 i,196, pairs changed 47
L==H
fullSet, iter:  0 i,197, pairs changed 47
L==H
fullSet, iter:  0 i,198, pairs changed 47
L==H
fullSet, iter:  0 i,199, pairs changed 47
L==H
fullSet, iter:  0 i,200, pairs changed 47
L==H
fullSet, iter:  0 i,201, pairs changed 47
L==H
fullSet, iter:  0 i,202, pairs changed 47
L==H
fullSet, iter:  0 i,203, pairs changed 47
L==H
fullSet, iter:  0 i,204, pairs changed 47
L==H
fullSet, iter:  0 i,205, pairs changed 47
L==H
fullSet, iter:  0 i,206, pairs changed 47
L==H
fullSet,

L==H
fullSet, iter:  0 i,366, pairs changed 47
L==H
fullSet, iter:  0 i,367, pairs changed 47
L==H
fullSet, iter:  0 i,368, pairs changed 47
L==H
fullSet, iter:  0 i,369, pairs changed 47
L==H
fullSet, iter:  0 i,370, pairs changed 47
L==H
fullSet, iter:  0 i,371, pairs changed 47
L==H
fullSet, iter:  0 i,372, pairs changed 47
L==H
fullSet, iter:  0 i,373, pairs changed 47
L==H
fullSet, iter:  0 i,374, pairs changed 47
L==H
fullSet, iter:  0 i,375, pairs changed 47
L==H
fullSet, iter:  0 i,376, pairs changed 47
L==H
fullSet, iter:  0 i,377, pairs changed 47
L==H
fullSet, iter:  0 i,378, pairs changed 47
L==H
fullSet, iter:  0 i,379, pairs changed 47
L==H
fullSet, iter:  0 i,380, pairs changed 47
L==H
fullSet, iter:  0 i,381, pairs changed 47
L==H
fullSet, iter:  0 i,382, pairs changed 47
L==H
fullSet, iter:  0 i,383, pairs changed 47
L==H
fullSet, iter:  0 i,384, pairs changed 47
L==H
fullSet, iter:  0 i,385, pairs changed 47
L==H
fullSet, iter:  0 i,386, pairs changed 47
L==H
fullSet,

L==H
fullSet, iter:  0 i,542, pairs changed 47
L==H
fullSet, iter:  0 i,543, pairs changed 47
L==H
fullSet, iter:  0 i,544, pairs changed 47
L==H
fullSet, iter:  0 i,545, pairs changed 47
L==H
fullSet, iter:  0 i,546, pairs changed 47
L==H
fullSet, iter:  0 i,547, pairs changed 47
L==H
fullSet, iter:  0 i,548, pairs changed 47
L==H
fullSet, iter:  0 i,549, pairs changed 47
L==H
fullSet, iter:  0 i,550, pairs changed 47
L==H
fullSet, iter:  0 i,551, pairs changed 47
L==H
fullSet, iter:  0 i,552, pairs changed 47
L==H
fullSet, iter:  0 i,553, pairs changed 47
L==H
fullSet, iter:  0 i,554, pairs changed 47
L==H
fullSet, iter:  0 i,555, pairs changed 47
L==H
fullSet, iter:  0 i,556, pairs changed 47
L==H
fullSet, iter:  0 i,557, pairs changed 47
L==H
fullSet, iter:  0 i,558, pairs changed 47
L==H
fullSet, iter:  0 i,559, pairs changed 47
L==H
fullSet, iter:  0 i,560, pairs changed 47
L==H
fullSet, iter:  0 i,561, pairs changed 47
L==H
fullSet, iter:  0 i,562, pairs changed 47
L==H
fullSet,

L==H
fullSet, iter:  0 i,719, pairs changed 47
L==H
fullSet, iter:  0 i,720, pairs changed 47
L==H
fullSet, iter:  0 i,721, pairs changed 47
L==H
fullSet, iter:  0 i,722, pairs changed 47
L==H
fullSet, iter:  0 i,723, pairs changed 47
L==H
fullSet, iter:  0 i,724, pairs changed 47
L==H
fullSet, iter:  0 i,725, pairs changed 47
L==H
fullSet, iter:  0 i,726, pairs changed 47
L==H
fullSet, iter:  0 i,727, pairs changed 47
L==H
fullSet, iter:  0 i,728, pairs changed 47
L==H
fullSet, iter:  0 i,729, pairs changed 47
L==H
fullSet, iter:  0 i,730, pairs changed 47
L==H
fullSet, iter:  0 i,731, pairs changed 47
L==H
fullSet, iter:  0 i,732, pairs changed 47
L==H
fullSet, iter:  0 i,733, pairs changed 47
L==H
fullSet, iter:  0 i,734, pairs changed 47
L==H
fullSet, iter:  0 i,735, pairs changed 47
L==H
fullSet, iter:  0 i,736, pairs changed 47
L==H
fullSet, iter:  0 i,737, pairs changed 47
L==H
fullSet, iter:  0 i,738, pairs changed 47
L==H
fullSet, iter:  0 i,739, pairs changed 47
L==H
fullSet,

...

j is not moving enough
fullSet, iter:  4764 i,1009, pairs changed 0
fullSet, iter:  4764 i,1010, pairs changed 0
fullSet, iter:  4764 i,1011, pairs changed 0
fullSet, iter:  4764 i,1012, pairs changed 0
fullSet, iter:  4764 i,1013, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1014, pairs changed 0
fullSet, iter:  4764 i,1015, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1016, pairs changed 0
fullSet, iter:  4764 i,1017, pairs changed 0
fullSet, iter:  4764 i,1018, pairs changed 0
fullSet, iter:  4764 i,1019, pairs changed 0
fullSet, iter:  4764 i,1020, pairs changed 0
fullSet, iter:  4764 i,1021, pairs changed 0
fullSet, iter:  4764 i,1022, pairs changed 0
fullSet, iter:  4764 i,1023, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1024, pairs changed 0
fullSet, iter:  4764 i,1025, pairs changed 0
fullSet, iter:  4764 i,1026, pairs changed 0
fullSet, iter:  4764 i,1027, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1028, p

j is not moving enough
fullSet, iter:  4764 i,1358, pairs changed 0
fullSet, iter:  4764 i,1359, pairs changed 0
fullSet, iter:  4764 i,1360, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1361, pairs changed 0
fullSet, iter:  4764 i,1362, pairs changed 0
fullSet, iter:  4764 i,1363, pairs changed 0
fullSet, iter:  4764 i,1364, pairs changed 0
fullSet, iter:  4764 i,1365, pairs changed 0
fullSet, iter:  4764 i,1366, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1367, pairs changed 0
fullSet, iter:  4764 i,1368, pairs changed 0
fullSet, iter:  4764 i,1369, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1370, pairs changed 0
fullSet, iter:  4764 i,1371, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1372, pairs changed 0
fullSet, iter:  4764 i,1373, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1374, pairs changed 0
fullSet, iter:  4764 i,1375, pairs changed 0
fullSet, iter:  4764 i,1376, pairs changed 0
fullSet

j is not moving enough
fullSet, iter:  4764 i,1538, pairs changed 0
fullSet, iter:  4764 i,1539, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1540, pairs changed 0
fullSet, iter:  4764 i,1541, pairs changed 0
fullSet, iter:  4764 i,1542, pairs changed 0
fullSet, iter:  4764 i,1543, pairs changed 0
fullSet, iter:  4764 i,1544, pairs changed 0
fullSet, iter:  4764 i,1545, pairs changed 0
fullSet, iter:  4764 i,1546, pairs changed 0
fullSet, iter:  4764 i,1547, pairs changed 0
fullSet, iter:  4764 i,1548, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1549, pairs changed 0
fullSet, iter:  4764 i,1550, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1551, pairs changed 0
fullSet, iter:  4764 i,1552, pairs changed 0
fullSet, iter:  4764 i,1553, pairs changed 0
fullSet, iter:  4764 i,1554, pairs changed 0
fullSet, iter:  4764 i,1555, pairs changed 0
fullSet, iter:  4764 i,1556, pairs changed 0
fullSet, iter:  4764 i,1557, pairs changed 0
fullSet,

j is not moving enough
fullSet, iter:  4764 i,1708, pairs changed 0
fullSet, iter:  4764 i,1709, pairs changed 0
fullSet, iter:  4764 i,1710, pairs changed 0
fullSet, iter:  4764 i,1711, pairs changed 0
fullSet, iter:  4764 i,1712, pairs changed 0
fullSet, iter:  4764 i,1713, pairs changed 0
fullSet, iter:  4764 i,1714, pairs changed 0
fullSet, iter:  4764 i,1715, pairs changed 0
fullSet, iter:  4764 i,1716, pairs changed 0
fullSet, iter:  4764 i,1717, pairs changed 0
fullSet, iter:  4764 i,1718, pairs changed 0
fullSet, iter:  4764 i,1719, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1720, pairs changed 0
fullSet, iter:  4764 i,1721, pairs changed 0
fullSet, iter:  4764 i,1722, pairs changed 0
fullSet, iter:  4764 i,1723, pairs changed 0
fullSet, iter:  4764 i,1724, pairs changed 0
fullSet, iter:  4764 i,1725, pairs changed 0
fullSet, iter:  4764 i,1726, pairs changed 0
fullSet, iter:  4764 i,1727, pairs changed 0
fullSet, iter:  4764 i,1728, pairs changed 0
fullSet, 

j is not moving enough
fullSet, iter:  4764 i,1843, pairs changed 0
L==H
fullSet, iter:  4764 i,1844, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1845, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1846, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1847, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1848, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1849, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1850, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1851, pairs changed 0
L==H
fullSet, iter:  4764 i,1852, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1853, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1854, pairs changed 0
L==H
fullSet, iter:  4764 i,1855, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1856, pairs changed 0
j is not moving enough
fullSet, iter:  4764 i,1857, pairs changed 0
j is not moving enough
fullSet, it